In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

from captum.attr import visualization as viz
from captum.attr import LayerConductance, LayerIntegratedGradients

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
from llvm_ml.torch import BasicBlockDataset
banned_ids = []
dataset = BasicBlockDataset("./data/ryzen3600_v16.cbuf", masked=True, banned_ids=banned_ids, prefilter=True)

In [4]:
model = torch.load("./data/bert_ryzen3600.pt")

In [5]:
print(model)

MCBERT(
  (embedding): MCEmbedding(
    (embedding): Embedding(21002, 256)
    (pos_encoding): PositionalEncoding(256)
    (norm): LayerNorm(256, affine=True, mode=graph)
  )
  (encoders): ModuleList(
    (0-3): 4 x MCGraphEncoder(
      (norm1): LayerNorm(256, affine=True, mode=graph)
      (attention): MCGraphAttention(
        (key): Linear(in_features=256, out_features=256, bias=False)
        (query): Linear(in_features=256, out_features=256, bias=False)
        (value): Linear(in_features=256, out_features=256, bias=False)
        (proj): Linear(in_features=256, out_features=256, bias=False)
      )
      (norm2): LayerNorm(256, affine=True, mode=graph)
      (feed_forward): Sequential(
        (0): Linear(in_features=256, out_features=512, bias=True)
        (1): GELU(approximate='none')
        (2): Linear(in_features=512, out_features=256, bias=True)
        (3): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (token_prediction): Linear(in_features=256, out_features=21002, b

In [6]:
model.eval()

MCBERT(
  (embedding): MCEmbedding(
    (embedding): Embedding(21002, 256)
    (pos_encoding): PositionalEncoding(256)
    (norm): LayerNorm(256, affine=True, mode=graph)
  )
  (encoders): ModuleList(
    (0-3): 4 x MCGraphEncoder(
      (norm1): LayerNorm(256, affine=True, mode=graph)
      (attention): MCGraphAttention(
        (key): Linear(in_features=256, out_features=256, bias=False)
        (query): Linear(in_features=256, out_features=256, bias=False)
        (value): Linear(in_features=256, out_features=256, bias=False)
        (proj): Linear(in_features=256, out_features=256, bias=False)
      )
      (norm2): LayerNorm(256, affine=True, mode=graph)
      (feed_forward): Sequential(
        (0): Linear(in_features=256, out_features=512, bias=True)
        (1): GELU(approximate='none')
        (2): Linear(in_features=512, out_features=256, bias=True)
        (3): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (token_prediction): Linear(in_features=256, out_features=21002, b

In [13]:
from torch_geometric.loader import DataLoader
loader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=6, drop_last=False)

model = model.to(device)
bb, raw, mask_id, original_token = next(iter(loader))
bb = bb.to(device)
model(bb.x, bb.edge_index, bb.batch)

tensor([[[ 1.9133e+01,  4.9177e-04,  2.5814e-04,  ...,  3.3463e-04,
          -1.8042e-05,  1.0846e-04],
         [ 6.7160e+00, -1.4771e-02, -1.4949e-02,  ..., -1.4980e-02,
          -1.4896e-02, -1.5038e-02],
         [ 6.5532e+00, -1.6451e-02, -1.6412e-02,  ..., -1.6821e-02,
          -1.6500e-02, -1.6619e-02],
         ...,
         [ 8.6527e+00, -2.3661e-02, -2.3680e-02,  ..., -2.4098e-02,
          -2.3864e-02, -2.3917e-02],
         [ 5.3790e+00, -6.6291e-03, -6.6509e-03,  ..., -6.9841e-03,
          -6.7611e-03, -7.0343e-03],
         [ 7.2830e+00, -1.1569e-02, -1.1838e-02,  ..., -1.1805e-02,
          -1.1808e-02, -1.2009e-02]]], device='cuda:0',
       grad_fn=<ViewBackward0>)